## Dependencies

In [1]:
from dataclasses import dataclass # for neural network construction
import pickle                     # to import MNIST data
import gzip                       # to import MNIST data
import random                     # to initialize weights and biases
import numpy as np                # for all needed math
from PIL import Image, ImageOps   # for image file processing
from time import time             # for performance measurement
print("Dependencies loaded")

Dependencies loaded


## Basic math functions

### Sigmoid function 

It is used as the activation function in all layers of our network and is defined as:

$$\sigma(z) = \frac{1}{1 + e^{-z}}$$

where $z$ is an `ndarray` (vector) and it will apply the formula element-wise and return another `ndarray` of the same shape as $z$.

In [2]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))
print("sigmoid defined")

sigmoid defined


### Derivative of the sigmoid function

Is used in the backpropagation algorithm and is defined as:

$$ \sigma^\prime(z) = \left(\frac{1}{1 + e^{-z}}\right)^\prime = \left((1 + e^{-z})^{-1}\right)^\prime $$

using the _power rule_ we can rewrite this as:

$$ -\frac{(1 + e^{-z})^\prime}{(1 + e^{-z})^{2}} $$

then using the _sum rule_ rewrite it as (taking the derivative of the nominator):

$$ -(1 + e^{-z})^{-2}(e^{-z})^\prime$$

applying the _exponential rule_ we get:

$$ -(1 + e^{-z})^{-2}e^{-z}(-z)^\prime $$ 

derivative of $-z$ is simply $-1$ that cancels the minus sign, so we now have:

$$ (1 + e^{-z})^{-2}e^{-z}$$

which is equivalent to:

$$ \frac{e^{-z}}{(1 + e^{-z})^{2}} $$ 

that can be rewritten as:

$$ \frac{1}{1 + e^{-z}}\frac{e^{-z}}{1 + e^{-z}}$$

and now with a sleight of hand (by adding and subtracting $1$) we can make it to be:

$$ \frac{1}{1 + e^{-z}}\frac{e^{-z} + 1 - 1}{1 + e^{-z}} = \frac{1}{1 + e^{-z}}\left(\frac{1 + e^{-z}}{1 + e^{-z}} - \frac{1}{1 + e^{-z}}\right)$$

and after simplifying it, we end up with the formula:

$$\frac{1}{1 + e^{-z}}\left(1 - \frac{1}{1 + e^{-z}}\right) = \sigma(z)(1 - \sigma(z)) $$

that the code below implements.

In [3]:
def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))
print("Derivation of sigmoid defined")

Derivation of sigmoid defined


### Derivative of the cost function

We use simple mean square error as our cost function:

$$ C(a_L) = \frac{1}{2}\|y(x)-a_L(x)\|^2 $$

where the cost function is for a _single_ input $x$, and $a_L$ is the activation function (i.e. output) of the last layer of the network.

And its derivative (which is used in the backpropagation algorithm) is defined as:

$$ \left[\frac{dC_x}{da_L}\right] \equiv C^\prime(a_L)$$

We can rewrite the cost function as:

$$ C(a_L) = \frac{1}{2}(y^2 - 2ya_L + a_L^2) $$

and therefore its derivative is:

$$ C^\prime(a_L) = \frac{1}{2}(2a_L - 2y) = a_L - y $$

Note that again $x, y,$ and $a_L$ are `ndarray`s.

In [4]:
def cost_derivative(output_activations, y):
    return (output_activations - y)
print("cost_derivative defined")

cost_derivative defined


## The neural network

The network is not implemented as a full-blown class, as it was in the original code. It is done this way to allow markdown annotations of each function (because there is no simple or elegant way to split the class implementation into multiple cells of the notebook). It is initialized with a list of numbers, each number representing the number of neurons in the corresponding layer of the network. 

`init_network` returns a _dataclass_ with three elements: num_layers, biases, and weights. The latter two are lists of `num_layers-1` elements of the `ndarray` type.

In [5]:
@dataclass
class Network:
    num_layers: int
    biases: list
    weights: list

def init_network(layers):
    
    return Network(
        len(layers),
        
        # input layer doesn't have biases
        [np.random.randn(y, 1) for y in layers[1:]],
        
        # there are no (weighted) connections into input layer or out of the output layer
        [np.random.randn(y, x) for x, y in zip(layers[:-1], layers[1:])]
    )
print("Network defined")

Network defined


### Forward propagation function

Calculates activation vector for each layer and returns the last activation vector as `ndarray`.

Parameters:

- `nn`: the neural network object returned by the `init_network` function.
- `a`: the network input as `ndarray`

In [6]:
def feedforward(nn, a):
    for b, w in zip(nn.biases, nn.weights):
        a = sigmoid(np.dot(w, a) + b)
    return a
print("Forward propagation defined")

Forward propagation defined


### Evaluation function 

Returns the number of test inputs for which the neural network outputs the correct result. Note that the neural    network's output is assumed to be the index of whichever neuron in the final layer has the highest activation (one-hot encoding).

Parameters:

- `nn`: the neural network object returned by the `init_network` function.
- `test_data`: a list of tuples (x, y) where x is the input and y is the correct output.

In [7]:
def evaluate(nn, test_data):
    test_results = [(np.argmax(feedforward(nn, x)), y) for (x, y) in test_data]
    
    return sum(int(x == y) for (x, y) in test_results)
print("Evaluation function defined")

Evaluation function defined


### Learning function

This is the main function, however the actual work is done by `stochastic_gradient_descent`, which this function invokes for each mini-batch of the training data.

Parameters:

- `nn`: the neural network object returned by the `init_network` function.
- `training_data`: a list of tuples (x, y) representing the training inputs and (known) correct outputs.
- `epochs`: number of epochs of training. During each epoch we go through all training data, but in a different order.
- `mini_batch_size`: the number of samples in a mini-batch.
- `learning_rate`: a hyper-parameter, 0.3 is the recommended value.
- `test_data`: another list of tuples (x, y) representing the test inputs and known outputs.

If `test_data` is provided then the network will be evaluated against the test data after each epoch, and partial progress printed out. This is useful for tracking progress, but slows things down.

In [8]:
def learn(nn, training_data, epochs, mini_batch_size, learning_rate, test_data = None):
    n = len(training_data)

    for j in range(epochs):
        random.shuffle(training_data) # that's where "stochastic" comes from

        mini_batches = [
            training_data[k: k + mini_batch_size] for k in range(0, n, mini_batch_size)
        ]
        
        for mini_batch in mini_batches:
            stochastic_gradient_descent(nn, mini_batch, learning_rate) # that's where learning really happes

        if test_data:
            print('Epoch {0}: accuracy {1}%'.format(f'{j + 1:2}', 100.0 * evaluate(nn, test_data) / len(test_data)))
        else:
            print('Epoch {0} complete.'.format(f'{j + 1:2}'))
print("Learning function defined")

Learning function defined


### Worker function


Updates the network's weights and biases by applying stochastic gradient descent using backpropagation to 
a single mini-batch according to these equations:

$$w = w^\prime - \frac{\eta}{m}\nabla{w}$$

$$b = b^\prime - \frac{\eta}{m}\nabla{b}$$

where $m$ is the number of samples in the mini-batch, and $\eta$ is the learning rate.

$\nabla{w}$ and $\nabla{b}$ are calculated by consequently adding $\delta\nabla{w}$ and $\delta\nabla{b}$ for each sample in the mini-batch, and $\delta\nabla{w}$ and $\delta\nabla{b}$ are computed by the backpropagation function.

Parameters:
- `nn`: the neural network object returned by the `init_network` function.
- `mini_batch`: a list of training data tuples (x, y) representing the training inputs and (known) correct outputs.
- `eta`: the learning rate hyper-parameter $\eta$.

In [9]:
def stochastic_gradient_descent(nn, mini_batch, eta):
    # "nabla" is the gradient symbol
    nabla_b = [np.zeros(b.shape) for b in nn.biases]
    nabla_w = [np.zeros(w.shape) for w in nn.weights]
    
    for x, y in mini_batch:
        delta_nabla_b, delta_nabla_w = backprop(nn, x, y) # compute the gradient
        
        # note that here we call the return values 'delta_nabla', while in the
        # backprop function we call them 'nabla'

        nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
        nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

        
    nn.weights = [w - (eta / len(mini_batch)) * nw for w, nw in zip(nn.weights, nabla_w)]
    nn.biases  = [b - (eta / len(mini_batch)) * nb for b, nb in zip(nn.biases, nabla_b)]
print("Worker defined")

Worker defined


### Backpropagation

Backpropagation function returns the gradient of the cost function:

$$\nabla{C_x} = \left[ \frac{\partial C_x}{\partial b}, \frac{\partial C_x}{\partial w} \right]$$

as a tuple $(\nabla{b}, \nabla{w})$. 

First we populate all $z$ and $a$ vectors for each layer during "feedforward" pass.

Then we go backwards in the network starting at the last layer $L$ and calculate:

$$\delta_L = C^\prime(z_L) $$

This derivative of the cost function $C$ can be expressed using _chain rule_ as:

$$ C^\prime(a(z))a^\prime(z) = \frac{dC}{da(z)}a^\prime(z)$$ 

note that

$$ a(z) \equiv \sigma(z) $$

and so $\delta$ can be calculated as follows (this is the same as equation **BP1** in the book):

$$ \delta_L = C^\prime(a_L) \circ \sigma^\prime(z_L) $$

where $\circ$ denotes Hadamard product. After that we can calculate gradient vector elements for weights and biases: 

$$ \nabla{b_L} = \delta_{L} \tag{equation BP3} $$

$$ \nabla{w_L} = \delta_L \cdot  a_{L-1}^\top \tag{equation BP4} $$

Note that the original equation **BP4** was written as:

$$ \frac{\partial{C}}{\partial{w^L_{jk}}} = a^{L-1}_k\delta^L_j $$

but we want the shape of $\nabla{w}$ to be the same as the shape of $w$, so we swap the operands while transposing the activation vector so it is now a row vector:

$$ \nabla{w} = \begin{bmatrix} 
                \delta_1 \\
                \delta_2 \\
                \vdots \\
                \delta_J \\
               \end{bmatrix} \cdot \begin{bmatrix} a_1 a_2 \dots a_K \end{bmatrix} $$

where $J$ is the number of neurons in the $L$th layer, and $K$ is the number of neurons in the $(L-1)$th layer. This multiplication will result in the matrix of $(J, K)$ shape, that is with $J$ rows and $K$ columns. Remember that in the book the weight matrix elements $w^l_{jk}$ are defined as connection weights from $k$th neuron in the $l-1$ layer to the $j$th neuron in the $l$th layer, so the weight matrix would be the same $(J, K)$ shape.

Then we recalculate $\delta$ for each _preceeding_ layer $i$, starting with $i = L - 1$:

$$ \delta_{i} = w_{i+1}^\top \cdot \delta_{i+1} \circ \sigma^\prime(z_i) \tag{equation BP2}$$

and using $\delta_i$ calculate new $\nabla{b_i}$ and $\nabla{w_i}$

Parameters:

- `nn`: the neural network object returned by the `init_network` function.
- `x`: input
- `y`: desired output

The function also calculates $z = w \cdot x + b$ and the activation $a = \sigma(z)$. $x, y, z, b, w$ are all `ndarrays`, and $w \cdot x$ is the dot product.

In [10]:
def backprop(nn, x, y):
    nabla_b = [np.zeros(b.shape) for b in nn.biases]
    nabla_w = [np.zeros(w.shape) for w in nn.weights]

    # feedforward
    activation = x    # first layer activation is just its input
    activations = [x] # list to store all activations, layer by layer
    zs = []           # list to store all z vectors, layer by layer

    for b, w in zip(nn.biases, nn.weights):
        z = np.dot(w, activation) + b  # calculate z for the current layer
        zs.append(z)                   # store
        activation = sigmoid(z)        # layer output
        activations.append(activation) # store

    # backward pass

    # 1. starting from the output layer
    delta = cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1]) 
    nabla_b[-1] = delta
    nabla_w[-1] = np.dot(delta, activations[-2].transpose())

    # 2. continue back to the input layer (i is the layer index, we're using i instead of l
    #    to improve readability -- l looks too much like 1)
    for i in range(2, nn.num_layers): # starting from the next-to-last layer
        z = zs[-i]
        sp = sigmoid_prime(z)
        delta = np.dot(nn.weights[-i + 1].transpose(), delta) * sp
        
        nabla_b[-i] = delta
        nabla_w[-i] = np.dot(delta, activations[-i - 1].transpose())
        
    return (nabla_b, nabla_w)
print("Backpropagation defined")

Backpropagation defined


## MNIST data

### Data load function

Returns the MNIST data as a tuple containing the training data, the validation data, and the test data.

The `training_data` is returned as a tuple with two entries. The first entry contains the actual training images.  This is an `ndarray` with 50,000 entries.  Each entry is, in turn, an `ndarray` with 784 values, representing the 28 * 28 = 784 pixels in a single MNIST image.

The second entry in the `training_data` tuple is an `ndarray` containing 50,000 entries.  Those entries are just the digit values (0...9) for the corresponding images contained in the first entry of the tuple.

The `validation_data` and `test_data` are similar, except each contains only 10,000 images.

This is a nice data format, but for use in neural networks it's helpful to modify the format of the `training_data` a little.

That's done in the wrapper function `load_data_wrapper()`, see below.

In [11]:
import re
from urllib import request
def load_data():
    url1 = 'https://raw.githubusercontent.com/tromgy/simple-neural-networks/master/mnist.pkl.gz'
    file_name1 = re.split(pattern='/', string=url1)[-1]
    r1 = request.urlretrieve(url=url1, filename=file_name1)
    txt1 = re.split(pattern=r'.', string=file_name1)[0] + ".txt"
    f = gzip.open(file_name1, 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="bytes")
    f.close()
    
    return (training_data, validation_data, test_data)
print("Loading of training data defined")

Loading of training data defined


### Data reshaper function

Returns a tuple containing `(training_data, validation_data, test_data)`. Based on `load_data`, but the format is more convenient for use in our implementation of neural networks.

In particular, `training_data` is a list containing 50,000 2-tuples `(x, y)`.  `x` is a 784-dimensional `ndarray`
containing the input image.  `y` is a 10-dimensional `ndarray` representing the unit vector corresponding to the
correct digit for `x`.

`validation_data` and `test_data` are lists containing 10,000 2-tuples `(x, y)` tuples. In each case, `x` is a 784-dimensional `ndarray` containing the input image, and `y` is the corresponding classification, i.e., the digit values (integers) corresponding to `x`.

Obviously, this means we're using slightly different formats for the training data and the validation / test data.  These formats turn out to be the most convenient for use in our neural network code.

In [12]:
def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [one_hot_encode(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    
    return (list(training_data), list(validation_data), list(test_data))
print("Data reshaper defined")

Data reshaper defined


### One-hot encoder

Returns a 10-dimensional unit vector with a 1.0 in the $j$th position and zeroes elsewhere.  This is used to convert a digit (0...9) into a corresponding desired output from the neural network.

Parameters:

- `j`: the index in the array to set to 1

In [13]:
def one_hot_encode(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    
    return e
print("one_hot_encode defined")

one_hot_encode defined


### Utility function

Prints the shape of a given `ndarray`. The first number is the number of rows, the second number is the number of columns.

Parameters:

- `name`: the name of the data
- `data`: the `ndarray` of data

In [14]:
def print_shape(name, data):
    print('Shape of {0}: {1}'.format(name, data.shape))
print("print_shape defined")

print_shape defined


## Main program

Create and train a simple network to recognize handwritten digits based on MNIST data.

We use 30 neurons in the intermediate layer and with the learning rate of 3.0, mini-batch size of 10, the network should achieve about 95% accuracy in 15 epochs. 

In [15]:
training_data, validation_data, test_data = load_data_wrapper() # load data

nn = init_network([784, 30, 10])

for l in range(0, nn.num_layers - 1):
    print('Network layer {0}'.format(l + 2)) # disregard the input layer
    print_shape('weights', nn.weights[l])
    print_shape('biases', nn.biases[l])
    
# hyper parameters
epochs = 15
mini_batch_size = 10
learning_rate = 3.0
    
print('Learning process started...')

time_start = time()

learn(nn, training_data, epochs, mini_batch_size, learning_rate, test_data)

time_end = time()

time_elapsed = time_end - time_start

print('Learning process complete in {0} seconds ({1} seconds per epoch)!'.format(f'{time_elapsed:.0f}', f'{time_elapsed / epochs:.1f}'))

print('Validation (with yet unseen data): accuracy {0}%'.format(100.0 * evaluate(nn, validation_data) / len(validation_data)))

Network layer 2
Shape of weights: (30, 784)
Shape of biases: (30, 1)
Network layer 3
Shape of weights: (10, 30)
Shape of biases: (10, 1)
Learning process started...


Epoch  1: accuracy 89.84%


Epoch  2: accuracy 92.22%


Epoch  3: accuracy 92.93%


Epoch  4: accuracy 92.7%


...

Epoch 12: accuracy 94.44%


Epoch 13: accuracy 94.57%


Epoch 14: accuracy 94.52%


Epoch 15: accuracy 94.55%
Learning process complete in 857 seconds (57.1 seconds per epoch)!


Validation (with yet unseen data): accuracy 94.83%


### Load image function

This function loads an image from a file and returns an `ndarray` of floats with [784x1] dimensions, the same as in the MNIST set.

The file is assumed to be 28x28 pixels, grayscale. _There is no error checking of any kind._

Parameters:

- `file_name`: the path to the image file

In [16]:
import requests
from io import BytesIO

def load_image(file_name):
    response = requests.get(file_name)
    digit = Image.open(BytesIO(response.content))
    
    # invert, so that background is black (zeros) 

    digit = ImageOps.invert(digit)
    
    pixels = digit.load()
    
    return np.array(digit).reshape((784, 1)) / 255
print("Function for loading image defined")

Function for loading image defined


### Image recognition function

This function takes a path template and the file name (which must be just a single-digit number corresponding to the digit image in the file), loads the image, and passes it through the neural network. The result is then compared with the file number.

Parameters:
- `path`: path template where the file name is inserted
- `file`: the number corresponding to the file name e.g. 1 => '1.png'

In [17]:
def recognize_image(path, file):
    x = load_image(path.format(file))

    y = feedforward(nn, x)
    
    bitmap = x.reshape((28, 28))
    
    file_num = int(file)
    result = y.argmax()
    
    if file_num == result:
        ev = 'correctly'
    else:
        ev = 'incorrectly'
 
    print(file, 'was', ev, 'recognized as', result)
print("Function for recognizing image defined")

Function for recognizing image defined


## Real-life test!

Load images of hand-written digits from our custom-made files and process them with our neural network.

In [18]:
print('Non-MNIST digits:')

for file in range(0,10):
    recognize_image('https://raw.githubusercontent.com/tromgy/simple-neural-networks/master/non-MNIST-digits/{0}.png', file)

Non-MNIST digits:
0 was incorrectly recognized as 2

1 was correctly recognized as 1


2 was correctly recognized as 2
3 was correctly recognized as 3


4 was correctly recognized as 4


5 was correctly recognized as 5
6 was correctly recognized as 6


7 was incorrectly recognized as 2


8 was incorrectly recognized as 5


9 was correctly recognized as 9
